<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Modélisation : Boosted Residual Tree</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">

# <b>1. Configuration du notebook</b>

In [46]:
# Imports des modules du projet
import config, src

# Imports classiques
import numpy as np
import pandas as pd

# Imports des modèles
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

# Imports pour l'optimisation
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Imports pour la visualisation
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Configuration d'un template par défaut pour la visualisation 
pio.templates['custom'] = pio.templates['plotly_white']
pio.templates['custom'].layout.colorway = px.colors.qualitative.Prism

pio.templates.default = 'custom'

---
# <b>2. Chargement et préparation des données</b>

In [3]:
# Chargement des données prétraitées
data = src.load_data(config.PROCESSED_DATA_FILE)

# Conversion des données au format datetime
data['Date'] = pd.to_datetime(data['Date'])

# Utilisation des dates comme index
data.set_index('Date', inplace=True)

# Ajout de la fréquence des données
data = data.asfreq('10min')

Données chargées avec succès.


In [4]:
# Variables explicatives
features = data.columns.drop('Energy')

# Données explicatives
X = data[features]

# Données cibles
y = data['Energy']

---
# <b>3. Définition du modèle</b>

In [5]:
# Taille de la fenêtre d'apprentissage séquentiel (2 mois)
window_size = (data.index <= "2016-03-11 17:00:00").sum()

# Taille des pas de l'apprentissage séquentiel (nombre de données à prédire)
step_size = 144 # Une journée

# Première fenêtre de l'apprentissage séquentiel
X_train_init, y_train_init = X.iloc[:window_size], y.iloc[:window_size]
X_test_init, y_test_init = X.iloc[window_size:window_size + step_size], y.iloc[window_size:window_size + step_size]

In [34]:
# Modèle de prédiction (Gradient Boosting)
xgb_model = XGBRegressor(random_state=1)

# Paramètres à optimiser du modèle de prédiction
xgb_param_grid = {
    'max_depth': [3, 5, 10, 15],
    "min_child_weight": [10, 20, 30, 40, 50],
    "gamma": [0, 0.1, 1, 10],
    "learning_rate": [0.001, 0.01, 0.01, 0.1],
    "n_estimators": [100, 300, 500],
    "subsample": [0.3, 0.4, 0.5],
    "colsample_bytree": [0.3, 0.7, 1.0],
    "lambda": [1, 10, 20, 50],
    "alpha": [1, 10, 20, 50]
}

# Modèle résiduel (Decision Tree)
dt_model = DecisionTreeRegressor(random_state=1)

# Paramètres à optimiser du modèle résiduel
dt_param_grid = {
    'max_depth': [2, 5, 10],
    'min_samples_split': [10, 20, 50, 100, 150, 200],
    'min_samples_leaf': [50, 100, 150, 200, 300, 500],
    'max_features': [7, 15, 25]
}

In [25]:
# Optimisation du modèle de prédiction
xgb_rsearch = RandomizedSearchCV(xgb_model, xgb_param_grid, n_iter=500, n_jobs=-1, cv=3, verbose=1, random_state=1)
xgb_rsearch.fit(X_train_init, y_train_init)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=1, ...),
                   n_iter=500, n_jobs=-1,
                   param_distributions={'alpha': [1, 10, 20, 50],
                                        'colsample_bytree': [0.3, 0.7, 1.0],
                                        'gamma': [0, 0.1, 1, 10],
                                        'lambda': [1, 10, 20, 50],
                                        'learning_rate': [0.001, 0.01, 0.01,
                                                          0.1],
                                        'max_depth': [3, 5, 10, 15],
                                        'min_child_weight': [10, 20, 30, 40,
                                                             50],
                                        'n_estimators': [100, 300, 500],
                                        'subsample': [0.3, 0.4, 0.5]},
                   random_state=1, verbose=1)

In [28]:
xgb_model_opt = xgb_rsearch.best_estimator_

In [35]:
# Optimisation du modèle résiduel
dt_rsearch = GridSearchCV(dt_model, dt_param_grid, n_jobs=-1, cv=3, verbose=1)
dt_rsearch.fit(X_train_init, y_train_init)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 10], 'max_features': [7, 15, 25],
                         'min_samples_leaf': [50, 100, 150, 200, 300, 500],
                         'min_samples_split': [10, 20, 50, 100, 150, 200]},
             verbose=1)

In [37]:
dt_model_opt = dt_rsearch.best_estimator_

---
# <b>4. Entraînement et évaluation du modèle</b>

In [38]:
results, metrics, metrics_evolution = src.online_window_fit(
    xgb_model_opt,
    dt_model_opt,
    X,
    y,
    window_size,
    step_size
)

1 | Apprentissage des données de 2016-01-11 17:00:00 à 2016-03-11 17:00:00 | Prévision de 2016-03-11 17:10:00 à 2016-03-12 17:00:00
2 | Apprentissage des données de 2016-01-12 17:00:00 à 2016-03-12 17:00:00 | Prévision de 2016-03-12 17:10:00 à 2016-03-13 17:00:00
3 | Apprentissage des données de 2016-01-13 17:00:00 à 2016-03-13 17:00:00 | Prévision de 2016-03-13 17:10:00 à 2016-03-14 17:00:00
4 | Apprentissage des données de 2016-01-14 17:00:00 à 2016-03-14 17:00:00 | Prévision de 2016-03-14 17:10:00 à 2016-03-15 17:00:00
5 | Apprentissage des données de 2016-01-15 17:00:00 à 2016-03-15 17:00:00 | Prévision de 2016-03-15 17:10:00 à 2016-03-16 17:00:00
6 | Apprentissage des données de 2016-01-16 17:00:00 à 2016-03-16 17:00:00 | Prévision de 2016-03-16 17:10:00 à 2016-03-17 17:00:00
7 | Apprentissage des données de 2016-01-17 17:00:00 à 2016-03-17 17:00:00 | Prévision de 2016-03-17 17:10:00 à 2016-03-18 17:00:00
8 | Apprentissage des données de 2016-01-18 17:00:00 à 2016-03-18 17:00:00 |

In [39]:
metrics

,MSE,RMSE,MAE,MAPE
0,8296.634592,91.085864,52.002911,0.579972


In [45]:
month_labels = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai'
}

period_begin = results.index.min()
period_end = results.index.max()

# Intervalle de dates pour l'axe temporel des affichages
date_range = pd.date_range(start=period_begin, end=period_end, freq='14D').strftime('%Y-%m-%d').tolist()
date_labels = [f'{pd.to_datetime(d).day} {month_labels[pd.to_datetime(d).month]}' for d in date_range]
fig = go.Figure()

fig.add_trace(go.Scatter(x=results.index, y=results['y_true'], mode='lines', name='Valeur réelle'))
fig.add_trace(go.Scatter(x=results.index, y=results['y_pred'], mode='lines', name='Prédiction', line=dict(color='red')))

fig.update_layout(
    title='Prédictions du modèle Boosted Residual Tree',
    xaxis_title='Dates',
    xaxis_tickvals=date_range,
    xaxis_ticktext=date_labels,
    yaxis_title='Consommation (en Wh)'
    )

fig.show()

In [44]:
for metric in metrics_evolution:
    fig = px.line(metrics_evolution, x=metrics_evolution.index, y=metric, title=f'Évolution de la {metric}')

    fig.update_layout(
        xaxis_title='Itérations',
        width=1000
    )

    fig.show()

---